# 4.2 Assistants


The Assistants API makes it possible to create AI assistants in applications. An Assistant has instructions and can use models, tools and files to answer the user's questions. The Assistants API currently supports three types of tools: Code Interpreter, File Search and Function Calling.


#

# 4.2.1 Assistant

In this example, an Assistant is created who is a personal maths tutor.

Initially, an Assistant is created, which represents an entity that can be configured to respond to a user's messages using various parameters such as a , *__name__*, *__instructions__* and *__model__*.

In [3]:

from openai import OpenAI
from runners.standard_run import std_run
from runners.streaming_run import streaming_run


client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    model="gpt-3.5-turbo",
)

Next, a Thread is created that represents a conversation between a user and one or more Assistants. A Thread can be created when a user (or their AI application) starts a conversation with their Assistant.

In [4]:
thread = client.beta.threads.create()

A message is added to the Thread, the content of the messages that users or applications create is added as objects to the Message Thread. 

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `8x + 12 = 14`. Can you help me?"
)

You need to create an execution, after all the user messages have been added to the Thread, you can execute the Thread with any assistant . Creating an Execution uses the model and tools associated with the assistant  to generate a response. These responses are added to the Thread as *__assistant__*  ard messages.

The run can be with streaming or without streaming.


In the run with streaming, the streaming_run function is defined, which is responsible for starting the streaming with the assistant. 
Within this is defined the 'EventHandler' class, which inherits from AssistantEventHandler. This class contains the methods for dealing with different events during streaming: 

* __'on_text_created':__ Called when a new text is created by the assistant. This prints 'assistant' to indicate that the assistant is responding.

* __'on_text_delta':__ Called for each text update. This prints the part of the text that was generated (delta.value).

* __'on_tool_call_created':__ Called when the wizard calls a tool. This prints the type of tool call (tool_call.type).

* __'on_tool_call_delta':__  Called for tool call updates, specifically for the code_interpreter. This prints the code_interpreter input and any outputs, especially logs.


Finally, the API client's create_and_stream method is used to start streaming, passing the EventHandler as the event handler.


* __'thread_id'__ and __'assistant_id'__ are the required identifiers.
* __'instructions'__ defines specific instructions for the assistant, in this case asking it to address the user as a SISTCA student.
* __'event_handler'__ is the instance of the EventHandler class.

The with block ensures that streaming continues until it is finished (stream.until_done()).

In [7]:
# With streaming

from typing_extensions import override
from openai import AssistantEventHandler


def streaming_run (thread_id, assistant_id, client):

    # First, we create a EventHandler class to define
    # how we want to handle the events in the response stream.
    
    class EventHandler(AssistantEventHandler):    
        @override
        def on_text_created(self, text) -> None:
            print(f"\nassistant > ", end="", flush=True)
            
        @override
        def on_text_delta(self, delta, snapshot):
            print(delta.value, end="", flush=True)
            
        def on_tool_call_created(self, tool_call):
            print(f"\nassistant > {tool_call.type}\n", flush=True)
        
        def on_tool_call_delta(self, delta, snapshot):
            if delta.type == 'code_interpreter':
                if delta.code_interpreter.input:
                    print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)


    # Then, we use the `create_and_stream` SDK helper 
    # with the `EventHandler` class to create the Run 
    # and stream the response.
    
    with client.beta.threads.runs.stream(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions="Please address the user as SISTCA student.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In run without streaming, you define the std_run function to start a new run in an existing thread, to wait for that run to complete and finally to list the messages in the thread and return the text of the first message if the run completes successfully, or return the run status otherwise.

In [6]:
# Without streaming

def std_run (thread_id, assistant_id, client):
    run = client.beta.threads.runs.create_and_poll(
        thread_id = thread_id,
        assistant_id = assistant_id,
        instructions = "Please address the user as SISTCA student."
    )   

    if run.status == 'completed': 

        messages = client.beta.threads.messages.list(
            thread_id = thread_id
        )

        return(messages.data[0].content[0].text.value)        

    else:
        return(run.status)

Finally, the solution is printed with the desired run type:

* __With streaming__

In [8]:
print(streaming_run(thread.id, assistant.id, client))


assistant > Of course, SISTCA student! To solve the equation 8x + 12 = 14, we first need to isolate the variable x. Here's how you can do it:

8x + 12 = 14

First, subtract 12 from both sides:

8x = 14 - 12
8x = 2

Now, divide both sides by 8 to solve for x:

x = 2/8
x = 1/4

So, the solution to the equation 8x + 12 = 14 is x = 1/4. If you have any other questions or need further clarification, feel free to ask!None


* __Without streaming__

In [7]:
print(std_run(thread.id, assistant.id, client))

Sure, SISTCA student! I can help you solve the equation `8x + 12 = 14`.

First, let's isolate the variable x by getting rid of the constant on the same side as x. We can do this by subtracting 12 from both sides of the equation:

8x + 12 - 12 = 14 - 12
8x = 2

Next, divide by 8 on both sides to solve for x:

8x/8 = 2/8
x = 0.25

So, the solution to the equation `8x + 12 = 14` is x = 0.25. If you have any more questions or need further assistance, feel free to ask!
